In [1]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import math
from nltk.stem import PorterStemmer
ps = PorterStemmer()


In [2]:
import bs4 as bs  
import urllib.request  
import re
scraped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Artificial_intelligence')  
article = scraped_data.read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:  
    article_text += p.text

In [3]:
def computeTFIDF(TF_scores, IDF_scores):
    TFIDF_scores = []
    for j in IDF_scores:
        for i in TF_scores:
            if j['key'] == i['key'] and j['doc_id'] == i['doc_id']:
                temp = {'doc_id' : j['doc_id'],
                        'TFIDF_score' : j['IDF_score' ]*i[ 'TF_score' ],
                        'key' : i['key']}
        TFIDF_scores.append(temp)
    return TFIDF_scores

In [4]:
def count_words(sent):
    count = 0
    words = word_tokenize(sent)
    for word in words:
        count += 1
    return count


In [5]:
def create_freq_dict(sents):
    i = 0
    freqDict_list = []
    for sent in sents:
        i += 1
        freq_dict = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in freq_dict:
                freq_dict[word] += 1
            else:
                freq_dict[word] = 1
            temp = {'doc_id' : i, 'freq_dict': freq_dict}
        freqDict_list.append(temp)
    return freqDict_list

In [6]:
def remove_string_special_characters(s):
    stripped = re.sub('[^\w\s]','', s)
    stripped = re.sub('_', '', stripped)
    stripped = re.sub('\s+', ' ', stripped)
    stripped = re.sub(r'\[[0-9]*\]', ' ', stripped)
    stripped = stripped.strip()
    return stripped

In [7]:
def get_doc(sent):
    doc_info = []
    i = 0
    for sent in text_sents_clean:
        i += 1
        count = count_words(sent)
        temp = {'doc_id' : i, 'doc_length' : count}
        doc_info.append(temp)
    return doc_info

In [8]:
def computeTF(doc_info, freqDict_list):
    TF_scores = []
    for tempDict in freqDict_list:
        id = tempDict['doc_id']
        for k in tempDict['freq_dict']:
            temp = {'doc_id' : id,
                    'TF_score' : tempDict['freq_dict'][k]/ doc_info[id-1]['doc_length'],
                    'key' : k}
            TF_scores.append(temp)
    return TF_scores

In [9]:
 def computeIDF(doc_info, freqDict_list):
        IDF_scores = []
        counter = 0
        for dict in freqDict_list:
            counter += 1
            for k in dict['freq_dict'].keys():
                count= sum([k in tempDict[ 'freq_dict'] for tempDict in freqDict_list])
                temp = {'doc_id' : counter, 'IDF_score' : math.log(len(doc_info)/count),'key' : k}
                IDF_scores.append( temp)
        return IDF_scores

In [10]:
text_sents = sent_tokenize(article_text)

In [11]:
text_sents_clean = [remove_string_special_characters(s) for s in text_sents]

In [12]:
doc_info = get_doc(text_sents_clean)

In [13]:
freqDict_list = create_freq_dict(text_sents_clean)

In [14]:
IDF_scores = computeIDF(doc_info, freqDict_list)

In [15]:
TF_scores = computeTF(doc_info, freqDict_list)

In [16]:
TFIDF_scores = computeTFIDF(TF_scores, IDF_scores)

In [17]:
TFIDF_scores

[{'doc_id': 1, 'TFIDF_score': 0.07469792908202781, 'key': 'in'},
 {'doc_id': 1, 'TFIDF_score': 0.08609639370421243, 'key': 'comput'},
 {'doc_id': 1, 'TFIDF_score': 0.1500405374637528, 'key': 'scienc'},
 {'doc_id': 1, 'TFIDF_score': 0.08353134083887889, 'key': 'artifici'},
 {'doc_id': 1, 'TFIDF_score': 0.2701006868593752, 'key': 'intellig'},
 {'doc_id': 1, 'TFIDF_score': 0.0475520851472683, 'key': 'ai'},
 {'doc_id': 1, 'TFIDF_score': 0.18894826483600202, 'key': 'sometim'},
 {'doc_id': 1, 'TFIDF_score': 0.15775864113537397, 'key': 'call'},
 {'doc_id': 1, 'TFIDF_score': 0.16958497574106476, 'key': 'machin'},
 {'doc_id': 1, 'TFIDF_score': 0.05112061335692085, 'key': 'is'},
 {'doc_id': 1, 'TFIDF_score': 0.173353452985688, 'key': 'demonstr'},
 {'doc_id': 1, 'TFIDF_score': 0.157727231138968, 'key': 'by'},
 {'doc_id': 1, 'TFIDF_score': 0.18036582055468628, 'key': 'contrast'},
 {'doc_id': 1, 'TFIDF_score': 0.026237621122668132, 'key': 'to'},
 {'doc_id': 1, 'TFIDF_score': 0.018190351518288166, '

In [18]:
import statistics
def get_sent_score(TFIDF_scores, text_sents, doc_info):
    sentence_info = []
    for doc in doc_info:
        sent_score = 0
        for i in range(0,len(TFIDF_scores)):
            temp_dict = TFIDF_scores[i]
            
            if doc['doc_id'] == temp_dict['doc_id']:
                sent_score += temp_dict['TFIDF_score']
        temp = {'doc_id' : doc['doc_id'], 'sent_score':sent_score,
                'sentence':text_sents[doc['doc_id']-1]}
        sentence_info.append(temp)
        
    return (sentence_info)

In [19]:
sentence_info = get_sent_score(TFIDF_scores, text_sents, doc_info)

In [20]:
def get_summary(sentence_info):
    sum = 0
    summary = []
    array = []
    
    for temp_dict in sentence_info:
        sum += temp_dict['sent_score']
        
    avg = (sum/0.8)/len(sentence_info)
    for temp_dict in sentence_info:
        array.append(temp_dict['sent_score'])
    
    stdev = statistics.stdev(array)
    
    for sent in sentence_info:
        if (sent['sent_score']) >= avg:
            summary.append(sent['sentence'])
    summary = '\n'.join(summary)
    return (summary)

In [21]:
summary = get_summary(sentence_info)

In [22]:
print(summary)

[2] A quip in Tesler's Theorem says "AI is whatever hasn't been done yet.
(Rossum's Universal Robots).
[29] Attendees Allen Newell (CMU), Herbert Simon (CMU), John McCarthy (MIT), Marvin Minsky (MIT) and Arthur Samuel (IBM) became the founders and leaders of AI research.
quiz show exhibition match, IBM's question answering system, Watson, defeated the two greatest Jeopardy!
champions, Brad Rutter and Ken Jennings, by a significant margin.
1 ranking for two years.
Clark also presents factual data indicating that error rates in image processing tasks have fallen significantly since 2011.
[131][132][133] Moderate successes related to affective computing include textual sentiment analysis and, more recently, multimodal affect analysis (see multimodal sentiment analysis), wherein AI classifies the affects displayed by a videotaped subject.
[180] Alternatively, distributed search processes can coordinate via swarm intelligence algorithms.
Early pioneers also include Alexey Grigorevich Ivakhn